In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import json
import e3nn
import numpy as np
import matplotlib.pyplot as plt 

import plotly.graph_objects as go
from plotly.subplots import make_subplots
from tabulate import tabulate
from IPython.display import HTML, display


sys.path.insert(1, "/mnt/arteml/playground/src/")
import viz
import tools

import h5py
import torch

DEVICE="cuda:0"

import pandas as pd
from pathlib import Path
import os
from tqdm.auto import trange

In [18]:
def process_mapper_stats(mapper_stats, print_flag=False):
    # print(mapper_stats)
    summary = mapper_stats[mapper_stats.index('Summary Stats'):]
    # print(summary)
    summary = summary[:summary.index('pJ/Compute')]
    # print(summary)

    cycle_index = summary.index('Cycles:')
    energy_index = summary.index('Energy:')
    area_index = summary.index('Area:')
    computes_index = summary.index('Computes')

    cycles = summary[cycle_index: ]
    cycles = cycles[cycles.index(' '):]
    cycles = int(cycles[: cycles.index('\n')].strip(' '))

    energy = summary[energy_index: ]
    energy = energy[energy.index(' '):].strip(' ')
    energy = float(energy[: energy.index(' ')])

    area = summary[area_index: ]
    area = area[area.index(' '):].strip(' ')
    area = float(area[: area.index(' ')])

    computes = summary[computes_index: ]
    computes = computes[computes.index(' '):].strip(' ')
    computes = int(computes[computes.rindex(' '):])

    if print_flag:
        print('Cycles:', cycles)
        print('Energy (in uJ):', energy)
        print('Area (in mm^2):', area)
        print('Computes:', computes)
    
    return cycles, energy, area, computes

# process_mapper_stats(mapper_stats, print_flag=True)

In [19]:

ARCHE_PATH  = Path("../../../example_designs/eyeriss_like")

model_architectures = ['AlexNet', 'U-Net', 'VGG01', 'VGG02']
stat_types = ['cycles', 'energies', 'areas', 'computes']

num_layers = {}

all_stats = {}
for model in model_architectures:
    all_stats[model] = {}
    for stat in stat_types:
        all_stats[model][stat] = []    

for i in trange(len(model_architectures)):
    model = model_architectures[i]
    
    layers = os.listdir(f"runs/eyeriss_{model}")
    
    num_layers[model] = 0
    for j in trange(len(layers)):
        if layers[j].startswith('.'):
            continue
        num_layers[model] += 1
        layer = layers[j]
        run_dirname = f"eyeriss_{model}/{layer}"
        run_path = Path("runs") / str(run_dirname)
        run_path.mkdir(parents=True, exist_ok=True)

        with open(f'{run_path}/timeloop-mapper.stats.txt', 'r') as file:
            mapper_stats = file.read()
                    
        cycles, energy, area, computes = process_mapper_stats(mapper_stats, print_flag=False)
                
        all_stats[model]['cycles'].append(cycles)
        all_stats[model]['energies'].append(energy)
        all_stats[model]['areas'].append(area)
        all_stats[model]['computes'].append(computes)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/14 [00:00<?, ?it/s]

In [20]:
model_architectures = ['AlexNet', 'U-Net', 'VGG01', 'VGG02']
stat_types = ['cycles', 'energies', 'areas', 'computes']
scales = {
        'cycles':1e7,
        'energies': 1e4,
        'areas': 1,
        'computes': 1e9
        }

mapping_ylabel = {'cycles': '10^7 x Cycles', 'energies': 'Energy in 10^-2 J', 'computes': '10^9 x Computes'}
mapping_title = {'cycles': '# of cycles comparison', 'energies': 'Energy consumption comparison', 'computes': '# of computes comparison'}

stat_types = ['cycles', 'energies', 'computes']

data = {}

x = np.arange(1, len(model_architectures) + 1)

for model in model_architectures:
    data[model] = {}

    for stat in stat_types:
        data[model][stat]= sum(all_stats[model][stat]) / len(all_stats[model][stat]) / num_layers[model]

In [21]:
data

{'AlexNet': {'cycles': 315428.16000000003,
  'energies': 277.3456,
  'computes': 43065365.760000005},
 'U-Net': {'cycles': 507921.9054820416,
  'energies': 729.2582608695653,
  'computes': 79091793.3005671},
 'VGG01': {'cycles': 696192.0,
  'energies': 798.89328125,
  'computes': 116960256.0},
 'VGG02': {'cycles': 418761.4674556213,
  'energies': 488.4019526627219,
  'computes': 68813187.78698225}}

In [14]:
import plotly.graph_objs as go
import plotly.io as pio

# Create bar traces for each parameter (energy, cycles, compute_per_j)

cb_friendly_colors = {
    "energies": "#1f77b4",
    "cycles": "#ff7f0e",
    "computes": "#2ca02c",
}
# grayscale_colors = {
#     "energies": "#666666",
#     "cycles": "#999999",
#     "computes": "#cccccc",
# }

traces = []
for metric in stat_types:
    traces.append(
        go.Bar(
            x=model_architectures,
            y=[(data[model][metric] / scales[metric]) for model in data],
            name=mapping_ylabel[metric],
            # marker_color=cb_friendly_colors[metric]
        )
    )

# "Cumulative metrics across all layers for different CNNs on the Eyeriss architecture"
# "Average metrics per layer for different CNNs on the Eyeriss architecture"

# Create a grouped layout
layout = go.Layout(
    title="Cumulative metrics across all layers for different CNNs on the Eyeriss architecture",
    barmode="group",
    template="plotly_white",
    xaxis=dict(title="Machine Learning Models"),
    yaxis=dict(title="Values"),
    width=800, height=400
)

# Create a Figure
fig = go.Figure(data=traces, layout=layout)

# Show the figure
pio.show(fig)


In [22]:

model_architectures = ['eyeriss', 'simba', 'simple_output_stationary', 'simple_weight_stationary']
stat_types = ['cycles', 'energies', 'areas', 'computes']

num_layers = {}

all_stats = {}
for model in model_architectures:
    all_stats[model] = {}
    for stat in stat_types:
        all_stats[model][stat] = []    

for i in trange(len(model_architectures)):
    model = model_architectures[i]
    
    layers = os.listdir(f"runs/{model}_U-Net")
    
    num_layers[model] = 0
    for j in trange(len(layers)):
        if layers[j].startswith('.'):
            continue
        num_layers[model] += 1
        layer = layers[j]
        run_dirname = f"{model}_U-Net/{layer}"
        run_path = Path("runs") / str(run_dirname)
        run_path.mkdir(parents=True, exist_ok=True)

        with open(f'{run_path}/timeloop-mapper.stats.txt', 'r') as file:
            mapper_stats = file.read()
                    
        cycles, energy, area, computes = process_mapper_stats(mapper_stats, print_flag=False)
                
        all_stats[model]['cycles'].append(cycles)
        all_stats[model]['energies'].append(energy)
        all_stats[model]['areas'].append(area)
        all_stats[model]['computes'].append(computes)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

In [23]:
stat_types = ['cycles', 'energies', 'areas', 'computes']
scales = {
        'cycles':1e7,
        'energies': 1e4,
        'areas': 1,
        'computes': 1e9
        }

mapping_ylabel = {'cycles': '10^7 x Cycles', 'energies': 'Energy in 10^-2 J', 'computes': '10^9 x Computes'}
mapping_title = {'cycles': '# of cycles comparison', 'energies': 'Energy consumption comparison', 'computes': '# of computes comparison'}

mapping_archs = {'eyeriss': 'Eyeriss-like',
                 'simba': 'Simba-like',
                 'simple_output_stationary': 'Output Stationary',
                 'simple_weight_stationary': 'Weight Stationary'
                 }
printable_arch_names = ['Eyeriss-like', 'Simba-like', 'Output Stationary', 'Weight Stationary']

stat_types = ['cycles', 'energies', 'computes']

data = {}

x = np.arange(1, len(model_architectures) + 1)

for model in model_architectures:
    data[model] = {}

    for stat in stat_types:
        data[model][stat]= sum(all_stats[model][stat]) / len(all_stats[model][stat])

In [24]:
import plotly.graph_objs as go
import plotly.io as pio

# Create bar traces for each parameter (energy, cycles, compute_per_j)

cb_friendly_colors = {
    "energies": "#1f77b4",
    "cycles": "#ff7f0e",
    "computes": "#2ca02c",
}
# grayscale_colors = {
#     "energies": "#666666",
#     "cycles": "#999999",
#     "computes": "#cccccc",
# }

traces = []
for metric in stat_types:
    traces.append(
        go.Bar(
            x=printable_arch_names,
            y=[(data[model][metric] / scales[metric]) for model in data],
            name=mapping_ylabel[metric],
            # marker_color=cb_friendly_colors[metric]
        )
    )

# "Cumulative metrics across all layers for different CNNs on the Eyeriss architecture"
# "Average metrics per layer for different CNNs on the Eyeriss architecture"

# Create a grouped layout
layout = go.Layout(
    title="Cumulative metrics across all layers for U-Net on different accelerator architectures",
    barmode="group",
    template="plotly_white",
    xaxis=dict(title="Accelerator Architecture"),
    yaxis=dict(title="Values"),
    width=800, height=400
)

# Create a Figure
fig = go.Figure(data=traces, layout=layout)

# Show the figure
pio.show(fig)


In [26]:

model_architectures = ['eyeriss', 'simba', 'simple_output_stationary', 'simple_weight_stationary']
stat_types = ['cycles', 'energies', 'areas', 'computes']

num_layers = {}

all_stats = {}
for model in model_architectures:
    all_stats[model] = {}
    for stat in stat_types:
        all_stats[model][stat] = []    

for i in trange(len(model_architectures)):
    model = model_architectures[i]
    
    layers = os.listdir(f"runs/{model}_U-Net")
    
    num_layers[model] = 0
    for j in trange(len(layers)):
        if layers[j].startswith('.'):
            continue
        num_layers[model] += 1
        layer = layers[j]
        run_dirname = f"{model}_U-Net/{layer}"
        run_path = Path("runs") / str(run_dirname)
        run_path.mkdir(parents=True, exist_ok=True)

        with open(f'{run_path}/timeloop-mapper.stats.txt', 'r') as file:
            mapper_stats = file.read()
                    
        cycles, energy, area, computes = process_mapper_stats(mapper_stats, print_flag=False)
                
        all_stats[model]['cycles'].append(cycles)
        all_stats[model]['energies'].append(energy)
        all_stats[model]['areas'].append(area)
        all_stats[model]['computes'].append(computes)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

In [27]:
stat_types = ['cycles', 'energies', 'areas', 'computes']
scales = {
        'cycles':1e7,
        'energies': 1e4,
        'areas': 1,
        'computes': 1e9
        }

mapping_ylabel = {'cycles': '10^7 x Cycles', 'energies': 'Energy in 10^-2 J', 'computes': '10^9 x Computes'}
mapping_title = {'cycles': '# of cycles comparison', 'energies': 'Energy consumption comparison', 'computes': '# of computes comparison'}

mapping_archs = {'eyeriss': 'Eyeriss-like',
                 'simba': 'Simba-like',
                 'simple_output_stationary': 'Output Stationary',
                 'simple_weight_stationary': 'Weight Stationary'
                 }
printable_arch_names = ['Eyeriss-like', 'Simba-like', 'Output Stationary', 'Weight Stationary']

stat_types = ['cycles', 'energies', 'computes']

data = {}

x = np.arange(1, len(model_architectures) + 1)

for model in model_architectures:
    data[model] = {}

    for stat in stat_types:
        data[model][stat]= sum(all_stats[model][stat]) / len(all_stats[model][stat])

In [48]:
import plotly.graph_objs as go
import plotly.io as pio


arches = ['eyeriss', 'simba', 'simple_output_stationary', 'simple_weight_stationary']

traces = []
for a in arches:
    data = all_stats[a]["cycles"]
    traces.append(
        go.Bar(
                x=list(range(1, len(data) + 1)),
                y=data,
                name=mapping_archs[a],
                # marker_color=cb_friendly_colors[metric]
            )

    )

# trace = go.Bar(x=list(range(1, len(data) + 1)),
#                 y=data
#                 name = 
#                 # marker_color=cb_friendly_colors[metric]
#             )
    

layout = go.Layout(
    title=f"Architecture, U-Net",
    template="plotly_white",
    barmode="group",
    xaxis=dict(title="Layer number"),
    yaxis=dict(title="# Cycles"),
    width=800, height=400
)

# Create a Figure
fig = go.Figure(data=traces, layout=layout)

# Show the figure
pio.show(fig)


In [45]:
import plotly.graph_objs as go
import plotly.io as pio


for arch in ['eyeriss', 'simba', 'simple_output_stationary', 'simple_weight_stationary']:
    data = all_stats[arch]["cycles"]

    trace = go.Bar(
                x=list(range(1, len(data) + 1)),
                y=data
                # marker_color=cb_friendly_colors[metric]
            )

    layout = go.Layout(
        title=f"{mapping_archs[arch]} architecture, U-Net",
        template="plotly_white",
        xaxis=dict(title="Layer number"),
        yaxis=dict(title="# Cycles"),
        width=800, height=400
    )

    # Create a Figure
    fig = go.Figure(data=trace, layout=layout)

    # Show the figure
    pio.show(fig)


In [ ]:
import plotly.graph_objs as go
import plotly.io as pio

data = all_stats["eyeriss"]["computes"]

trace = go.Bar(
            x=list(range(1, len(data) + 1)),
            y=data
            # marker_color=cb_friendly_colors[metric]
        )

layout = go.Layout(
    title="Cumulative metrics across all layers for U-Net on different accelerator architectures",
    template="plotly_white",
    xaxis=dict(title="Accelerator Architecture"),
    yaxis=dict(title="Values"),
    width=800, height=400
)

# Create a Figure
fig = go.Figure(data=trace, layout=layout)

# Show the figure
pio.show(fig)
